In [1]:
!pip3 install transformers datasets evaluate

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
eli5 = load_dataset("eli5_category", split="train[:5000]")

/Users/tanmaymahindrakar/Library/Python/3.9/lib/python/site-packages/datasets/load.py:1486: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Split the dataset’s train split into a train and test set

In [4]:
eli5 = eli5.train_test_split(test_size=0.2)

In [5]:
eli5["train"][0]

{'q_id': '5o3p78',
 'title': 'Why is horse racing such a tossup?',
 'selftext': "I'm wondering what it is about horse racing that makes it so consistently ideal for gambling. To clarify, I'm not asking why people use horse racing for gambling, that answer is evident (because its never a sure thing). More I'm curious about why it's never a sure thing to begin with. Why is there so much variance to a horse running a short distance in a controlled environment?",
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dcgdot6'],
  'text': ["They are set up that way. Track stewards assign weights to horses to even out the odds. A very good horse must carry more weight. The less weight the faster the horse runs. More weight also means more injuries. The horse's trainer weighs all of these factors when choosing what races to enter. The goal is to win every race. So showing off how fast a horse is at the beginning of the season will result in more weight being added ear

## load a DistilGPT2 tokenizer to process the text subfield

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

In [7]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '5o3p78',
 'title': 'Why is horse racing such a tossup?',
 'selftext': "I'm wondering what it is about horse racing that makes it so consistently ideal for gambling. To clarify, I'm not asking why people use horse racing for gambling, that answer is evident (because its never a sure thing). More I'm curious about why it's never a sure thing to begin with. Why is there so much variance to a horse running a short distance in a controlled environment?",
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dcgdot6'],
 'answers.text': ["They are set up that way. Track stewards assign weights to horses to even out the odds. A very good horse must carry more weight. The less weight the faster the horse runs. More weight also means more injuries. The horse's trainer weighs all of these factors when choosing what races to enter. The goal is to win every race. So showing off how fast a horse is at the beginning of the season will result in more weight being added 

In [8]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1547 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1241 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1259 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1322 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1356 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1128 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2815 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1331 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/tanmaymahindrakar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Currently logged in as: tanmay-mandy. Use `wandb login --relogin` to force relogin


  0%|          | 0/3930 [00:00<?, ?it/s]

{'loss': 3.978, 'grad_norm': 4.634866237640381, 'learning_rate': 1.7455470737913488e-05, 'epoch': 0.38}
{'loss': 3.9602, 'grad_norm': 4.238131523132324, 'learning_rate': 1.4910941475826972e-05, 'epoch': 0.76}


  0%|          | 0/326 [00:00<?, ?it/s]

{'eval_loss': 3.8367133140563965, 'eval_runtime': 37.9749, 'eval_samples_per_second': 68.624, 'eval_steps_per_second': 8.585, 'epoch': 1.0}
{'loss': 3.9102, 'grad_norm': 3.9605700969696045, 'learning_rate': 1.236641221374046e-05, 'epoch': 1.15}
{'loss': 3.8553, 'grad_norm': 3.7502660751342773, 'learning_rate': 9.821882951653945e-06, 'epoch': 1.53}
{'loss': 3.8579, 'grad_norm': 4.291957378387451, 'learning_rate': 7.27735368956743e-06, 'epoch': 1.91}


  0%|          | 0/326 [00:00<?, ?it/s]

{'eval_loss': 3.8251278400421143, 'eval_runtime': 31.6804, 'eval_samples_per_second': 82.259, 'eval_steps_per_second': 10.29, 'epoch': 2.0}
{'loss': 3.8311, 'grad_norm': 4.007526397705078, 'learning_rate': 4.732824427480917e-06, 'epoch': 2.29}
{'loss': 3.8175, 'grad_norm': 4.298675537109375, 'learning_rate': 2.1882951653944023e-06, 'epoch': 2.67}


  0%|          | 0/326 [00:00<?, ?it/s]

{'eval_loss': 3.824284315109253, 'eval_runtime': 33.5545, 'eval_samples_per_second': 77.665, 'eval_steps_per_second': 9.716, 'epoch': 3.0}
{'train_runtime': 1986.0938, 'train_samples_per_second': 15.824, 'train_steps_per_second': 1.979, 'train_loss': 3.8787625300671915, 'epoch': 3.0}


TrainOutput(global_step=3930, training_loss=3.8787625300671915, metrics={'train_runtime': 1986.0938, 'train_samples_per_second': 15.824, 'train_steps_per_second': 1.979, 'total_flos': 1026504284700672.0, 'train_loss': 3.8787625300671915, 'epoch': 3.0})

In [33]:
trainer.save_model("/Users/tanmaymahindrakar/causal_language_model")

In [35]:
model.save_pretrained("/Users/tanmaymahindrakar/causal_language_model_01")

In [16]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/326 [00:00<?, ?it/s]

Perplexity: 45.80


# Testing

In [17]:
prompt = "Somatic hypermutation allows the immune system to"

In [44]:
from transformers import pipeline

generator = pipeline("text-generation")
generator(prompt)

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/tanmaymahindrakar/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Somatic hypermutation allows the immune system to adapt to a more common form of infections such as breast cancer.\n\nThe study also found that women receiving an IV of the type needed to carry the new strain's DNA in a child can have"}]

In [43]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("/Users/tanmaymahindrakar/causal_language_model")